# Parsing Data
In this notebook, let's explore how to leverage generative AI to build and consume a knowledge graph in Neo4j.

This notebook parses Form-13 data from SEC EDGAR. This is partially structured data, a mix of text and XML.  Instead of spending our time writing a bespoke parser to extract data from these files and load into Neo4j, we can prompt a Large Language Model (LLM) to do this for us automatically.  We will then also use the LLM to generate Cypher statements to load the extracted data into a Neo4j graph.

## Setup
First, let's install the libraries we're going to need for this lab and the following notebook dependent labs.  We'll also want to reboot the kernel once done.  To do that, go to the "Kernel" menu and click "Restart Kernel and Clear All Outputs."  That will get rid of everything the install statements printed, leaving us with a cleaner notebook to work with.

In [ ]:
%pip install --user graphdatascience
%pip install --user "pydantic==1.10.11"
%pip install --user "langchain==0.1.20"
%pip install --user gradio
%pip install --user IProgress
%pip install --user tqdm
%pip install --user langchain-community
%pip install --user langchain-google-vertexai

# Restart the kernel after installing libraries
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Now restart the kernel. That will allow the Python evironment to import the new packages.

## Prompt Definition
We will extract knowledge adhering to the same schema we used previously.  To teach the LLM about the schema, we will use a series of prompts.  Each prompt is focused on only one task, extracting a specific entity:

1. Manager Information
2. Filing Information

## Functions for Using LLMs
Let's create some helper function to talk to the LLM with our prompt and text input. 

The [Vertex AI documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models) describes the available foundation models.  We will use the text-bison base model. In some cases, there may be a need to fine-tune LLM models for KG creation. [Vertex AI provides an elegant way to fine-tune](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models) where the updated weights/model stay within your tenant and the base model is frozen.

### Step 1: Ingesting Station Data
Let's ingest all of the data we need from the stations table and create nodes in our graph. First well define a template for our prompts. 

In [ ]:
station_info_tpl = """The input file below is a CSV file. Could you please convert it to JSON?

Text:
$ctext
"""

In [ ]:
station_info_tpl = """The input file below is a CSV file. The first line represents the column names. Each row after that represents a transport station in London.
For each transport station, extract the following entities from every row and put them into JSON format. Do not miss any of this information.
* "station_name" - This name appears in the "Station_Name" column of the CSV file. 
* "latitude" - This value appears in the "Latitude" column of the source file.
* "longitude" - This value appears in the "Longitude" column of the source file.
* "postcode" - This value appears in the "Postcode" column of the source file. 
* "zone" - This value appears in the "Zone" column of the source file. 
* "zone_original - This column is outdated, please ignore.
* Please convert this CSV file into a list of JSON object enclosed by 3 backticks. No other text in the response
* Please include every row of the CSV file in the JSON output.

Text:
$ctext
"""

Now we connect with the Google Cloud Storage bucket and read in the data from the CSV file containing the station data

In [ ]:
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.bucket('neo4j-datasets')
blob = bucket.blob('dataflow-london-transport/bigquery-to-neo4j/source-data/London_stations.csv')

raw_station_data = blob.download_as_string().decode()

We're going to need to break up this file into smaller chunks in order to stay under the rate limit quote for the LLM. The function below will read in the CSV file and will reuse the header row to generate a sequence of smaller CSV "files" to send sequentially to the language model. 

In [ ]:
# This function will split the CSV file into smaller chunks in order to stay under the LLM rate limit
def split_csv_file(inp_text, chunk_size=10):
    csv_chunks = []
    split_text = inp_text.split('\n')
    header_row = split_text[0]
    
    chunk = ''
    row_count = 0
    for row in split_text:
        if row_count == 0:
            chunk = header_row + '\r\n'
            row_count += 1
        if row != header_row:
            chunk += row + '\r\n'
            row_count += 1
        if row_count >= chunk_size+1:
            csv_chunks.append(chunk)
            row_count = 0
    csv_chunks.append(chunk)
    
    return csv_chunks
    

In [ ]:
csv_chunks = split_csv_file(raw_station_data, chunk_size=10)
print(csv_chunks[0])

Now let's define a couple more functions to call the language model

In [ ]:
from vertexai.language_models import TextGenerationModel

# Wrapper for calling language model
def run_text_model(
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    prompt: str,
    tuned_model_name: str = None,
    ) :
    """Text Completion Use a Large Language Model."""
    if tuned_model_name is None:
        model = TextGenerationModel.from_pretrained(model_name)
    else:
        model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        prompt,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    return response.text

In [ ]:
# Wrapper for entity extraction and parsing
def extract_entities_relationships(prompt, tuned_model_name=None):
    try:
        res = run_text_model("text-bison@001", 0, 1024, 0.8, 1, prompt, tuned_model_name)
        return res
    except Exception as e:
        print(e)

In [ ]:
import vertexai

vertexai.init()

In [ ]:
from string import Template

prompt_stations = Template(station_info_tpl).substitute(ctext=csv_chunks[0])
print(prompt_stations)

In [ ]:
import json

# Use LLM to parse out manager information
#station_data = extract_entities_relationships(prompt_stations).split('```')[1].strip('json')
station_data = extract_entities_relationships(prompt_stations)
station_data


Let's clean up this output a bit

In [ ]:
import json
import pprint
london_stations = json.loads(station_data)
pprint.pprint(london_stations)

#### Establish Neo4j Connection

In [ ]:
# username is neo4j by default
NEO4J_USERNAME = 'neo4j'

# You will need to change these to match your credentials
NEO4J_URI = 'neo4j+s://f3af8fc6.databases.neo4j.io:7687'
NEO4J_PASSWORD = 'neo4jneo4j123!'

In [ ]:
from graphdatascience import GraphDataScience

gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=True
)
gds.set_database('neo4j')

To merge the data, we can use parameterized Cypher queries.  Basically, we will send filings in batches (in this sample case, just one batch) for each node and relationship type and insert them as parameters in the query.

In [ ]:
# Merge company nodes
gds.run_cypher('''
UNWIND $records AS record
MERGE (s:Station {latitude:record.latitude, longitude:record.longitude, postocde:record.postcode, zone: record.zone})
SET s.name = record.station_name
RETURN count(s) AS company_node_merge_count
''', params={'records':london_stations})

### Step 2: Connecting The Stations by Transit Lines
Let's ingest all of the data we need from the stations table and create nodes in our graph. First well define a template for our prompts. 

In [ ]:
tube_line_info_tpl = """The input file below is a CSV file. The first line represents the column names. Extract the following entities from every row and put them into JSON format. Do not miss any of this information.
* "tube_line" - This name appears in the "Tube_Line" column of the CSV file. 
* "from_station" - This value appears in the "From_Station" column of the source file.
* "to_station" - This value appears in the "To_Station" column of the source file.
* Please convert this CSV file into a list of JSON objects.

Text:
$ctext
"""

In [ ]:
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.bucket('neo4j-datasets')
blob = bucket.blob('dataflow-london-transport/bigquery-to-neo4j/source-data/London_tube_lines.csv')

inp_text = blob.download_as_string().decode()
tube_line_chunks = split_csv_file(inp_text, chunk_size=20)
tube_line_chunks[0]

In [ ]:
import json
import pprint
london_stations = json.loads(station_data)
pprint.pprint(london_stations)

In [ ]:
from string import Template

prompt_tube_lines = Template(tube_line_info_tpl).substitute(ctext=tube_line_chunks[0])
print(prompt_tube_lines)

In [ ]:
prompt = Template(tube_line_info_tpl).substitute(ctext=tube_line_chunks[0])
#response = json.loads(extract_entities_relationships(prompt).split('```')[1].strip('json'))
#tube_line_info = extract_entities_relationships(prompt)
tube_line_info = json.loads(extract_entities_relationships(prompt))
print(tube_line_info)

## Parse Filing Information
We will parse filing info in a similar manner to manager information. Because the filings include a list of many entries however, we will want to split the input into chunks so as not to exceed input or output token limits. 

## Test Example

Let's walk through the steps to do this with just the 1 form above first, then we can move on to parsing and ingesting multiple form13s

To start we can run the LLM parsing over all the filing info from the form and then combine the resulting JSON into a list conducive for Neo4j loading.

In [ ]:
filings_list = []
import time
for filing_info_chunk in filing_info_chunks:
    prompt = Template(filing_info_tpl).substitute(ctext=filing_info_chunk)
    response = extract_entities_relationships(prompt)
    # time.sleep(2) #uncomment this line if you face any rate limit error
    if '```' in response:
        response = response.split('```')[1].strip('json')
    filings_list.extend(json.loads(response))

for item in filings_list:
    item['managerName'] = manager_data['managerName']
    item['reportCalendarOrQuarter'] = manager_data['reportCalendarOrQuarter']
filings_list[:5]

In [ ]:
len(filings_list)

In [ ]:
# Merge manager node
gds.run_cypher('''
MERGE (m:Manager {managerName: $name})
RETURN count(m) AS manager_node_merge_count
''', params={'name':manager_data['managerName']})

In [ ]:
# Merge owns Relationship
gds.run_cypher('''
UNWIND $records AS record
MATCH (m:Manager {managerName: record.managerName})
MATCH (c:Company {cusip: record.cusip})
MERGE(m)-[r:OWNS]->(c)
SET r.reportCalendarOrQuarter = record.reportCalendarOrQuarter,
    r.value = record.value,
    r.shares = record.shares
RETURN count(r) AS owns_relationship_merge_count
''', params={'records':filings_list})

## Ingest Multiple Form 13 Files
We will make a pipeline using the methods above.  In this case we will take a two-step approach, first parse all the data, then chunk that data and ingest into Neo4j.

For purposes of this lab we will just use a few form13 files.

In [ ]:
### If you have time to parse more files, you can uncomment these lines.
sample_file_names = [
   'hands-on-lab/form13-raw/raw_2022-01-03_archives_edgar_data_1844571_0001844571-22-000001.txt',
   'hands-on-lab/form13-raw/raw_2022-01-03_archives_edgar_data_1875995_0001875995-22-000004.txt',
   'hands-on-lab/form13-raw/raw_2022-01-06_archives_edgar_data_1495703_0001495703-22-000002.txt'
]

In [ ]:
# Helper function for getting filing info
def get_manager_and_filing_info(raw_txt):
    contents = raw_txt.split('<XML>')
    manager_info = contents[1].split('</XML>')[0].strip()
    filing_info = contents[2].split('</XML>')[0].strip()
    
    return manager_info, filing_info

In [ ]:
%%time

print(f'=== Parsing {len(sample_file_names)} Form 13 Files ===')

filings_list = []
manager_list = []

for file_name in sample_file_names:
    
    print(f'--- parsing {file_name} ---')
    try:
        # Get raw form13 file
        print('getting file text from gcloud....')
        blob = bucket.blob(file_name)
        raw_text = blob.download_as_string().decode()

        # Get raw manager and filing info from file
        print('getting file contents...')
        manager_info, filing_info = get_manager_and_filing_info(raw_text)

        # Parse manager info into dict using LLM
        print('Parsing submission and manager info...')
        mng_prompt = Template(mgr_info_tpl).substitute(ctext=manager_info)
        mng_response = extract_entities_relationships(mng_prompt)
        manager_data = json.loads(mng_response.replace('```', ''))
        manager_list.append({'managerName': manager_data['managerName']})

        # Parse filing info into list of dicts using LLM
        print('Parsing filing info...')
        tmp_filing_list = []
        for filing_info_chunk in split_filing_info(filing_info):
            filing_prompt = Template(filing_info_tpl).substitute(ctext=filing_info_chunk)
            filing_response = extract_entities_relationships(filing_prompt)
            #time.sleep(3) #uncomment this line if you face any rate limit error
            if '```' in filing_response:
                filing_response = filing_response.split('```')[1].strip('json')
            tmp_filing_list.extend(json.loads(filing_response))
        for item in tmp_filing_list: #Add information from manager_info to enable OWNS relationship loading
            item['managerName'] = manager_data['managerName']
            item['reportCalendarOrQuarter'] = manager_data['reportCalendarOrQuarter']
        filings_list.extend(tmp_filing_list)
    except Exception as e:
        print(filing_response)
        raise e


Now we can merge the mananger nodes

In [ ]:
# Merge manager nodes
gds.run_cypher('''
UNWIND $records AS record
MERGE (m:Manager {managerName: record.managerName})
RETURN count(m) AS manager_node_merge_count
''', params={'records':manager_list})

For filings lets check ther length of the list

In [ ]:
len(filings_list)

While we should not need chunking for this example, below is an example of how to chunk up a parameterized function for loading in case you need to scale up. 

In [ ]:
# As the dataset gets bigger we will want to chunk up the filings we send to Neo4j
def chunks(xs, n=10_000):
    n = max(1, n)
    return [xs[i:i + n] for i in range(0, len(xs), n)]

In [ ]:
# Merge company nodes
for d in chunks(filings_list):
    res = gds.run_cypher('''
    UNWIND $records AS record
    MERGE (c:Company {cusip: record.cusip})
    SET c.companyName = record.companyName
    RETURN count(c) AS company_node_merge_count
    ''', params={'records':d})
    print(res)

In [ ]:
# Merge owns Relationships
for d in chunks(filings_list):
    res = gds.run_cypher('''
    UNWIND $records AS record
    MATCH (m:Manager {managerName: record.managerName})
    MATCH (c:Company {cusip: record.cusip})
    MERGE(m)-[r:OWNS]->(c)
    SET r.reportCalendarOrQuarter = record.reportCalendarOrQuarter,
        r.value = record.value,
        r.shares = record.shares
    RETURN count(r) AS owns_relationship_merge_count
    ''', params={'records':d})
    print(res)

This type of workflow can be applied to other unstructured data to parse entities and relationships with language models and load them into a Neo4j knowledge graph. 